In [ ]:
import json
import requests
import os
import base64
import sklearn

from eval_cus import check
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
import torch
import pickle
import random
from collections import defaultdict

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
# from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.utils.linear_assignment_ import linear_assignment
import pandas as pd

def nmi(y_true, y_pred):
    return normalized_mutual_info_score(y_true, y_pred)


def ari(y_true, y_pred):
    return adjusted_rand_score(y_true, y_pred)


def acc(y_true, y_pred):
    y_true, y_pred = pd.Series(list(y_true)), pd.Series(list(y_pred))
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
# k = requests.post(url, data={"api_key": key, "api_secret": secret})
# print(k.text)

In [ ]:
        
# face_token = json.loads(k.text)["facealbum_token"]

In [ ]:
def sim(runs):
    #add image randomly
    secret = "" 
    key = ""
    faces_all = []
    acc_all = []
    nmi_all = []
    ari_all = []
    
    for run in range(runs):
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/createalbum"
        k = requests.post(url, data={"api_key": key, "api_secret": secret})
        face_token = json.loads(k.text)["facealbum_token"]
        print(f'==> Create Album count {run} \n{k.text}')
        
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/addimage"
        
        path = "custom_adv3"
        imageid2ground = {}
        images_stored = []
        no_faces = []
        faces = []

        persons = os.listdir(path)
        ground = 0
        print(f'==> Adding randomised images {run}')
        for g, person in enumerate(persons):
            if person.startswith('.'):
                continue

        #     print(ground, person)
            co = 0
            images = os.listdir(path + '/' + person)
            random.shuffle(images)
            
            print(f'curr:{ground} person:{person}')
            for image in images:
                print(f'curr:{ground} person:{person} co:{co}')
                if co >= 10:
                    break
                with open(path + '/' + person + '/'+ image, "rb") as img_file:
                    send_img64 = base64.b64encode(img_file.read())
                response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "image_base64":send_img64})
        #         print(response.text)
                if not response.ok:
                    print(response.text)
                    no_faces.append(path + '/' + person + '/'+ image)
                    continue
                else:
                    faces.append(path + '/' + person + '/'+ image)

                imgid = json.loads(response.text)["image_id"]
                imageid2ground[imgid] = ground
                images_stored.append(str(path + '/' + person + '/'+ image))
                co += 1
            ground += 1
            
        faces_all.append(faces)
        print(f'==> Grouping faces {run} ')
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/groupface"
        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
        print(response.text)
        task_id = json.loads(response.text)["task_id"]
        
        print(f'==> Getting clusters {run} ')
        start_token = ''
        predicted_clusters = []
        ground_truth = []
        url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"

        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
        start_token = json.loads(response.text)["next_token"]
        processed = defaultdict(lambda: False)
        

        repeat = defaultdict(int)

        while(True):
            print(len(json.loads(response.text)["faces"]))

            for j in json.loads(response.text)["faces"]:
                if not processed[j["image_id"]]:
                    predicted_clusters.append(int(j["group_id"]))
                    ground_truth.append(imageid2ground[j["image_id"]])
                    processed[j["image_id"]] = True
                else:
                    repeat[j["image_id"]] += 1

            try:
                start_token = json.loads(response.text)["next_token"]
            except:
                print(response.text)
                break

            response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token": start_token})
        acc_all.append(acc(ground_truth, predicted_clusters))
        nmi_all.append(nmi(ground_truth, predicted_clusters))
        ari_all.append(ari(ground_truth, predicted_clusters))
        
    
    pkl_file=open('faces_all.txt','wb')
    pickle.dump(faces_all, pkl_file)
    print(acc_all)
    print(nmi_all)
    print(ari_all)
    return acc_all, nmi_all, ari_all, faces_all, ground_truth, predicted_clusters

In [ ]:
acc, nmi, ari, faces_all, ground_truth, predicted_clusters = sim(10)

In [ ]:
acc, nmi, ari



In [ ]:
faces_all
# 

In [ ]:
print(zip(faces_all[0]), ground_truth, predicted_clusters)

In [ ]:
for j in zip(faces_all[0], ground_truth, predicted_clusters):
    print(j)

In [ ]:
t = [('custom_adv3/yaleB12/462.jpeg', 0, 0)
('custom_adv3/yaleB12/503.jpeg', 0, 0)
('custom_adv3/yaleB12/197.jpeg', 0, 1)
('custom_adv3/yaleB12/166.jpeg', 0, 1)
('custom_adv3/yaleB12/532.jpeg', 0, 1)
('custom_adv3/yaleB12/327.jpeg', 0, 1)
('custom_adv3/yaleB12/492.jpeg', 0, 1)
('custom_adv3/yaleB12/505.jpeg', 0, 1)
('custom_adv3/yaleB12/494.jpeg', 0, 1)
('custom_adv3/yaleB12/560.jpeg', 0, 1)
('custom_adv3/yaleB18/281.jpeg', 1, 0)
('custom_adv3/yaleB18/457.jpeg', 1, 0)
('custom_adv3/yaleB18/216.jpeg', 1, 0)
('custom_adv3/yaleB18/465.jpeg', 1, 0)
('custom_adv3/yaleB18/45.jpeg', 1, 0)
('custom_adv3/yaleB18/407.jpeg', 1, 0)
('custom_adv3/yaleB18/146.jpeg', 1, 23)
('custom_adv3/yaleB18/2.jpeg', 1, 0)
('custom_adv3/yaleB18/570.jpeg', 1, 0)
('custom_adv3/yaleB18/16.jpeg', 1, 23)
('custom_adv3/yaleB33/462.jpeg', 2, 6)
('custom_adv3/yaleB33/202.jpeg', 2, 6)
('custom_adv3/yaleB33/503.jpeg', 2, 0)
('custom_adv3/yaleB33/197.jpeg', 2, 6)
('custom_adv3/yaleB33/166.jpeg', 2, 0)
('custom_adv3/yaleB33/532.jpeg', 2, 6)
('custom_adv3/yaleB33/248.jpeg', 2, 0)
('custom_adv3/yaleB33/327.jpeg', 2, 6)
('custom_adv3/yaleB33/492.jpeg', 2, 6)
('custom_adv3/yaleB33/505.jpeg', 2, 6)
('custom_adv3/yaleB21/202.jpeg', 3, 9)
('custom_adv3/yaleB21/197.jpeg', 3, 9)
('custom_adv3/yaleB21/248.jpeg', 3, 9)
('custom_adv3/yaleB21/327.jpeg', 3, 0)
('custom_adv3/yaleB21/494.jpeg', 3, 9)
('custom_adv3/yaleB21/560.jpeg', 3, 24)
('custom_adv3/yaleB21/559.jpeg', 3, 24)
('custom_adv3/yaleB21/281.jpeg', 3, 0)
('custom_adv3/yaleB21/304.jpeg', 3, 9)
('custom_adv3/yaleB21/378.jpeg', 3, 0)
('custom_adv3/yaleB39/503.jpeg', 4, 0)
('custom_adv3/yaleB39/197.jpeg', 4, 1)
('custom_adv3/yaleB39/166.jpeg', 4, 0)
('custom_adv3/yaleB39/532.jpeg', 4, 0)
('custom_adv3/yaleB39/327.jpeg', 4, 1)
('custom_adv3/yaleB39/505.jpeg', 4, 1)
('custom_adv3/yaleB39/494.jpeg', 4, 1)
('custom_adv3/yaleB39/560.jpeg', 4, 1)
('custom_adv3/yaleB39/339.jpeg', 4, 0)
('custom_adv3/yaleB39/559.jpeg', 4, 1)
('custom_adv3/yaleB24/462.jpeg', 5, 0)
('custom_adv3/yaleB24/197.jpeg', 5, 1)
('custom_adv3/yaleB24/505.jpeg', 5, 1)
('custom_adv3/yaleB24/233.jpeg', 5, 0)
('custom_adv3/yaleB24/494.jpeg', 5, 1)
('custom_adv3/yaleB24/560.jpeg', 5, 1)
('custom_adv3/yaleB24/339.jpeg', 5, 0)
('custom_adv3/yaleB24/559.jpeg', 5, 1)
('custom_adv3/yaleB24/304.jpeg', 5, 1)
('custom_adv3/yaleB24/53.jpeg', 5, 0)
('custom_adv3/yaleB22/197.jpeg', 6, 0)
('custom_adv3/yaleB22/166.jpeg', 6, 0)
('custom_adv3/yaleB22/532.jpeg', 6, 0)
('custom_adv3/yaleB22/248.jpeg', 6, 0)
('custom_adv3/yaleB22/327.jpeg', 6, 1)
('custom_adv3/yaleB22/494.jpeg', 6, 1)
('custom_adv3/yaleB22/560.jpeg', 6, 1)
('custom_adv3/yaleB22/559.jpeg', 6, 1)
('custom_adv3/yaleB22/502.jpeg', 6, 0)
('custom_adv3/yaleB22/281.jpeg', 6, 0)
('custom_adv3/yaleB20/197.jpeg', 7, 1)
('custom_adv3/yaleB20/327.jpeg', 7, 1)
('custom_adv3/yaleB20/494.jpeg', 7, 1)
('custom_adv3/yaleB20/560.jpeg', 7, 1)
('custom_adv3/yaleB20/559.jpeg', 7, 1)
('custom_adv3/yaleB20/281.jpeg', 7, 0)
('custom_adv3/yaleB20/304.jpeg', 7, 0)
('custom_adv3/yaleB20/53.jpeg', 7, 0)
('custom_adv3/yaleB20/457.jpeg', 7, 1)
('custom_adv3/yaleB20/564.jpeg', 7, 1)
('custom_adv3/yaleB23/197.jpeg', 8, 1)
('custom_adv3/yaleB23/492.jpeg', 8, 0)
('custom_adv3/yaleB23/505.jpeg', 8, 1)
('custom_adv3/yaleB23/494.jpeg', 8, 1)
('custom_adv3/yaleB23/560.jpeg', 8, 1)
('custom_adv3/yaleB23/559.jpeg', 8, 1)
('custom_adv3/yaleB23/281.jpeg', 8, 0)
('custom_adv3/yaleB23/304.jpeg', 8, 1)
('custom_adv3/yaleB23/325.jpeg', 8, 0)
('custom_adv3/yaleB23/457.jpeg', 8, 1)
('custom_adv3/yaleB13/197.jpeg', 9, 14)
('custom_adv3/yaleB13/532.jpeg', 9, 14)
('custom_adv3/yaleB13/248.jpeg', 9, 14)
('custom_adv3/yaleB13/494.jpeg', 9, 0)
('custom_adv3/yaleB13/560.jpeg', 9, 1)
('custom_adv3/yaleB13/559.jpeg', 9, 1)
('custom_adv3/yaleB13/281.jpeg', 9, 0)
('custom_adv3/yaleB13/304.jpeg', 9, 1)
('custom_adv3/yaleB13/53.jpeg', 9, 0)
('custom_adv3/yaleB13/325.jpeg', 9, 0)
('custom_adv3/yaleB29/197.jpeg', 10, 11)
('custom_adv3/yaleB29/494.jpeg', 10, 0)
('custom_adv3/yaleB29/559.jpeg', 10, 0)
('custom_adv3/yaleB29/325.jpeg', 10, 0)
('custom_adv3/yaleB29/457.jpeg', 10, 0)
('custom_adv3/yaleB29/393.jpeg', 10, 11)
('custom_adv3/yaleB29/216.jpeg', 10, 0)
('custom_adv3/yaleB29/465.jpeg', 11, 8)
('custom_adv3/yaleB29/45.jpeg', 11, 0)
('custom_adv3/yaleB29/241.jpeg', 11, 8)
('custom_adv3/yaleB25/462.jpeg', 11, 8)
('custom_adv3/yaleB25/202.jpeg', 11, 1)
('custom_adv3/yaleB25/503.jpeg', 11, 1)
('custom_adv3/yaleB25/197.jpeg', 12, 13)
('custom_adv3/yaleB25/166.jpeg', 12, 13)
('custom_adv3/yaleB25/492.jpeg', 12, 13)
('custom_adv3/yaleB25/233.jpeg', 12, 0)
('custom_adv3/yaleB25/494.jpeg', 12, 10)
('custom_adv3/yaleB25/560.jpeg', 12, 0)
('custom_adv3/yaleB25/559.jpeg', 12, 0)
('custom_adv3/yaleB30/202.jpeg', 13, 0)
('custom_adv3/yaleB30/197.jpeg', 13, 20)
('custom_adv3/yaleB30/166.jpeg', 13, 0)
('custom_adv3/yaleB30/327.jpeg', 13, 0)
('custom_adv3/yaleB30/233.jpeg', 13, 0)
('custom_adv3/yaleB30/494.jpeg', 13, 0)
('custom_adv3/yaleB30/560.jpeg', 14, 0)
('custom_adv3/yaleB30/77.jpeg', 14, 12)
('custom_adv3/yaleB30/559.jpeg', 14, 0)
('custom_adv3/yaleB30/281.jpeg', 14, 12)
('custom_adv3/yaleB35/197.jpeg', 14, 0)
('custom_adv3/yaleB35/166.jpeg', 15, 3)
('custom_adv3/yaleB35/492.jpeg', 15, 3)
('custom_adv3/yaleB35/494.jpeg', 15, 3)
('custom_adv3/yaleB35/560.jpeg', 15, 3)
('custom_adv3/yaleB35/559.jpeg', 15, 3)
('custom_adv3/yaleB35/281.jpeg', 16, 0)
('custom_adv3/yaleB35/304.jpeg', 16, 0)
('custom_adv3/yaleB35/352.jpeg', 16, 5)
('custom_adv3/yaleB35/260.jpeg', 17, 1)
('custom_adv3/yaleB15/503.jpeg', 17, 1)
('custom_adv3/yaleB15/197.jpeg', 17, 22)
('custom_adv3/yaleB15/166.jpeg', 17, 1)
('custom_adv3/yaleB15/327.jpeg', 18, 0)
('custom_adv3/yaleB15/492.jpeg', 18, 21)
('custom_adv3/yaleB15/505.jpeg', 18, 0)
('custom_adv3/yaleB15/161.jpeg', 18, 0)
('custom_adv3/yaleB15/494.jpeg', 19, 1)
('custom_adv3/yaleB15/560.jpeg', 19, 1)
('custom_adv3/yaleB15/339.jpeg', 19, 0)
('custom_adv3/yaleB27/462.jpeg', 19, 1)
('custom_adv3/yaleB27/202.jpeg', 20, 2)
('custom_adv3/yaleB27/503.jpeg', 20, 2)
('custom_adv3/yaleB27/197.jpeg', 21, 1)
('custom_adv3/yaleB27/532.jpeg', 21, 0)
('custom_adv3/yaleB27/492.jpeg', 21, 1)
('custom_adv3/yaleB27/182.jpeg', 21, 1)
('custom_adv3/yaleB27/505.jpeg', 22, 19)
('custom_adv3/yaleB27/494.jpeg', 22, 17)
('custom_adv3/yaleB27/560.jpeg', 22, 0)
('custom_adv3/yaleB34/197.jpeg', 23, 0)
('custom_adv3/yaleB34/327.jpeg', 24, 15)
('custom_adv3/yaleB34/492.jpeg', 24, 15)
('custom_adv3/yaleB34/505.jpeg', 25, 4)
('custom_adv3/yaleB34/494.jpeg', 25, 4)
('custom_adv3/yaleB34/560.jpeg', 25, 4)
('custom_adv3/yaleB34/559.jpeg', 26, 7)
('custom_adv3/yaleB34/304.jpeg', 26, 0)
('custom_adv3/yaleB34/325.jpeg', 27, 0)
('custom_adv3/yaleB34/260.jpeg', 27, 1)
('custom_adv3/yaleB11/197.jpeg', 27, 0)
('custom_adv3/yaleB11/166.jpeg', 27, 1)

In [ ]:
([0.5324675324675324,
  0.5202702702702703,
  0.5033557046979866,
  0.4972067039106145,
  0.4701492537313433,
  0.4642857142857143,
  0.5389221556886228,
  0.5350877192982456,
  0.4369747899159664,
  0.4774774774774775],
 [0.648631240624919,
  0.6398877269623577,
  0.6330309083702043,
  0.6251953684792773,
  0.6109350790769701,
  0.6164700942357043,
  0.6749943717831609,
  0.6555659278807516,
  0.5475465427994163,
  0.6152822846511867],
 [0.11653929186197665,
  0.10951437056110923,
  0.1194167363067528,
  0.09654443016351577,
  0.1123183643380059,
  0.125195271416752,
  0.13315787486782926,
  0.15570203136366817,
  0.0642040965909668,
  0.12725762986124206])

In [ ]:
best attack: 0.34302, 0.5083, 0.06398

In [ ]:
f = ['custom_adv3/yaleB12/462.jpeg',
  'custom_adv3/yaleB12/503.jpeg',
  'custom_adv3/yaleB12/197.jpeg',
  'custom_adv3/yaleB12/166.jpeg',
  'custom_adv3/yaleB12/532.jpeg',
  'custom_adv3/yaleB12/327.jpeg',
  'custom_adv3/yaleB12/492.jpeg',
  'custom_adv3/yaleB12/505.jpeg',
  'custom_adv3/yaleB12/494.jpeg',
  'custom_adv3/yaleB12/560.jpeg',
  'custom_adv3/yaleB18/281.jpeg',
  'custom_adv3/yaleB18/457.jpeg',
  'custom_adv3/yaleB18/216.jpeg',
  'custom_adv3/yaleB18/465.jpeg',
  'custom_adv3/yaleB18/45.jpeg',
  'custom_adv3/yaleB18/407.jpeg',
  'custom_adv3/yaleB18/146.jpeg',
  'custom_adv3/yaleB18/2.jpeg',
  'custom_adv3/yaleB18/570.jpeg',
  'custom_adv3/yaleB18/16.jpeg',
  'custom_adv3/yaleB33/462.jpeg',
  'custom_adv3/yaleB33/202.jpeg',
  'custom_adv3/yaleB33/503.jpeg',
  'custom_adv3/yaleB33/197.jpeg',
  'custom_adv3/yaleB33/166.jpeg',
  'custom_adv3/yaleB33/532.jpeg',
  'custom_adv3/yaleB33/248.jpeg',
  'custom_adv3/yaleB33/327.jpeg',
  'custom_adv3/yaleB33/492.jpeg',
  'custom_adv3/yaleB33/505.jpeg',
  'custom_adv3/yaleB21/202.jpeg',
  'custom_adv3/yaleB21/197.jpeg',
  'custom_adv3/yaleB21/248.jpeg',
  'custom_adv3/yaleB21/327.jpeg',
  'custom_adv3/yaleB21/494.jpeg',
  'custom_adv3/yaleB21/560.jpeg',
  'custom_adv3/yaleB21/559.jpeg',
  'custom_adv3/yaleB21/281.jpeg',
  'custom_adv3/yaleB21/304.jpeg',
  'custom_adv3/yaleB21/378.jpeg',
  'custom_adv3/yaleB39/503.jpeg',
  'custom_adv3/yaleB39/197.jpeg',
  'custom_adv3/yaleB39/166.jpeg',
  'custom_adv3/yaleB39/532.jpeg',
  'custom_adv3/yaleB39/327.jpeg',
  'custom_adv3/yaleB39/505.jpeg',
  'custom_adv3/yaleB39/494.jpeg',
  'custom_adv3/yaleB39/560.jpeg',
  'custom_adv3/yaleB39/339.jpeg',
  'custom_adv3/yaleB39/559.jpeg',
  'custom_adv3/yaleB24/462.jpeg',
  'custom_adv3/yaleB24/197.jpeg',
  'custom_adv3/yaleB24/505.jpeg',
  'custom_adv3/yaleB24/233.jpeg',
  'custom_adv3/yaleB24/494.jpeg',
  'custom_adv3/yaleB24/560.jpeg',
  'custom_adv3/yaleB24/339.jpeg',
  'custom_adv3/yaleB24/559.jpeg',
  'custom_adv3/yaleB24/304.jpeg',
  'custom_adv3/yaleB24/53.jpeg',
  'custom_adv3/yaleB22/197.jpeg',
  'custom_adv3/yaleB22/166.jpeg',
  'custom_adv3/yaleB22/532.jpeg',
  'custom_adv3/yaleB22/248.jpeg',
  'custom_adv3/yaleB22/327.jpeg',
  'custom_adv3/yaleB22/494.jpeg',
  'custom_adv3/yaleB22/560.jpeg',
  'custom_adv3/yaleB22/559.jpeg',
  'custom_adv3/yaleB22/502.jpeg',
  'custom_adv3/yaleB22/281.jpeg',
  'custom_adv3/yaleB20/197.jpeg',
  'custom_adv3/yaleB20/327.jpeg',
  'custom_adv3/yaleB20/494.jpeg',
  'custom_adv3/yaleB20/560.jpeg',
  'custom_adv3/yaleB20/559.jpeg',
  'custom_adv3/yaleB20/281.jpeg',
  'custom_adv3/yaleB20/304.jpeg',
  'custom_adv3/yaleB20/53.jpeg',
  'custom_adv3/yaleB20/457.jpeg',
  'custom_adv3/yaleB20/564.jpeg',
  'custom_adv3/yaleB23/197.jpeg',
  'custom_adv3/yaleB23/492.jpeg',
  'custom_adv3/yaleB23/505.jpeg',
  'custom_adv3/yaleB23/494.jpeg',
  'custom_adv3/yaleB23/560.jpeg',
  'custom_adv3/yaleB23/559.jpeg',
  'custom_adv3/yaleB23/281.jpeg',
  'custom_adv3/yaleB23/304.jpeg',
  'custom_adv3/yaleB23/325.jpeg',
  'custom_adv3/yaleB23/457.jpeg',
  'custom_adv3/yaleB13/197.jpeg',
  'custom_adv3/yaleB13/532.jpeg',
  'custom_adv3/yaleB13/248.jpeg',
  'custom_adv3/yaleB13/494.jpeg',
  'custom_adv3/yaleB13/560.jpeg',
  'custom_adv3/yaleB13/559.jpeg',
  'custom_adv3/yaleB13/281.jpeg',
  'custom_adv3/yaleB13/304.jpeg',
  'custom_adv3/yaleB13/53.jpeg',
  'custom_adv3/yaleB13/325.jpeg',
  'custom_adv3/yaleB29/197.jpeg',
  'custom_adv3/yaleB29/494.jpeg',
  'custom_adv3/yaleB29/559.jpeg',
  'custom_adv3/yaleB29/325.jpeg',
  'custom_adv3/yaleB29/457.jpeg',
  'custom_adv3/yaleB29/393.jpeg',
  'custom_adv3/yaleB29/216.jpeg',
  'custom_adv3/yaleB29/465.jpeg',
  'custom_adv3/yaleB29/45.jpeg',
  'custom_adv3/yaleB29/241.jpeg',
  'custom_adv3/yaleB25/462.jpeg',
  'custom_adv3/yaleB25/202.jpeg',
  'custom_adv3/yaleB25/503.jpeg',
  'custom_adv3/yaleB25/197.jpeg',
  'custom_adv3/yaleB25/166.jpeg',
  'custom_adv3/yaleB25/492.jpeg',
  'custom_adv3/yaleB25/233.jpeg',
  'custom_adv3/yaleB25/494.jpeg',
  'custom_adv3/yaleB25/560.jpeg',
  'custom_adv3/yaleB25/559.jpeg',
  'custom_adv3/yaleB30/202.jpeg',
  'custom_adv3/yaleB30/197.jpeg',
  'custom_adv3/yaleB30/166.jpeg',
  'custom_adv3/yaleB30/327.jpeg',
  'custom_adv3/yaleB30/233.jpeg',
  'custom_adv3/yaleB30/494.jpeg',
  'custom_adv3/yaleB30/560.jpeg',
  'custom_adv3/yaleB30/77.jpeg',
  'custom_adv3/yaleB30/559.jpeg',
  'custom_adv3/yaleB30/281.jpeg',
  'custom_adv3/yaleB35/197.jpeg',
  'custom_adv3/yaleB35/166.jpeg',
  'custom_adv3/yaleB35/492.jpeg',
  'custom_adv3/yaleB35/494.jpeg',
  'custom_adv3/yaleB35/560.jpeg',
  'custom_adv3/yaleB35/559.jpeg',
  'custom_adv3/yaleB35/281.jpeg',
  'custom_adv3/yaleB35/304.jpeg',
  'custom_adv3/yaleB35/352.jpeg',
  'custom_adv3/yaleB35/260.jpeg',
  'custom_adv3/yaleB15/503.jpeg',
  'custom_adv3/yaleB15/197.jpeg',
  'custom_adv3/yaleB15/166.jpeg',
  'custom_adv3/yaleB15/327.jpeg',
  'custom_adv3/yaleB15/492.jpeg',
  'custom_adv3/yaleB15/505.jpeg',
  'custom_adv3/yaleB15/161.jpeg',
  'custom_adv3/yaleB15/494.jpeg',
  'custom_adv3/yaleB15/560.jpeg',
  'custom_adv3/yaleB15/339.jpeg',
  'custom_adv3/yaleB27/462.jpeg',
  'custom_adv3/yaleB27/202.jpeg',
  'custom_adv3/yaleB27/503.jpeg',
  'custom_adv3/yaleB27/197.jpeg',
  'custom_adv3/yaleB27/532.jpeg',
  'custom_adv3/yaleB27/492.jpeg',
  'custom_adv3/yaleB27/182.jpeg',
  'custom_adv3/yaleB27/505.jpeg',
  'custom_adv3/yaleB27/494.jpeg',
  'custom_adv3/yaleB27/560.jpeg',
  'custom_adv3/yaleB34/197.jpeg',
  'custom_adv3/yaleB34/327.jpeg',
  'custom_adv3/yaleB34/492.jpeg',
  'custom_adv3/yaleB34/505.jpeg',
  'custom_adv3/yaleB34/494.jpeg',
  'custom_adv3/yaleB34/560.jpeg',
  'custom_adv3/yaleB34/559.jpeg',
  'custom_adv3/yaleB34/304.jpeg',
  'custom_adv3/yaleB34/325.jpeg',
  'custom_adv3/yaleB34/260.jpeg',
  'custom_adv3/yaleB11/197.jpeg',
  'custom_adv3/yaleB11/166.jpeg',
  'custom_adv3/yaleB11/494.jpeg',
  'custom_adv3/yaleB11/560.jpeg',
  'custom_adv3/yaleB11/559.jpeg',
  'custom_adv3/yaleB11/281.jpeg',
  'custom_adv3/yaleB11/304.jpeg',
  'custom_adv3/yaleB11/18.jpeg',
  'custom_adv3/yaleB11/325.jpeg',
  'custom_adv3/yaleB11/260.jpeg',
  'custom_adv3/yaleB38/503.jpeg',
  'custom_adv3/yaleB38/197.jpeg',
  'custom_adv3/yaleB38/560.jpeg',
  'custom_adv3/yaleB38/559.jpeg',
  'custom_adv3/yaleB38/281.jpeg',
  'custom_adv3/yaleB38/304.jpeg',
  'custom_adv3/yaleB38/413.jpeg',
  'custom_adv3/yaleB38/53.jpeg',
  'custom_adv3/yaleB38/325.jpeg',
  'custom_adv3/yaleB38/562.jpeg',
  'custom_adv3/yaleB36/197.jpeg',
  'custom_adv3/yaleB36/166.jpeg',
  'custom_adv3/yaleB36/494.jpeg',
  'custom_adv3/yaleB36/559.jpeg',
  'custom_adv3/yaleB36/53.jpeg',
  'custom_adv3/yaleB36/260.jpeg',
  'custom_adv3/yaleB36/457.jpeg',
  'custom_adv3/yaleB36/216.jpeg',
  'custom_adv3/yaleB36/234.jpeg',
  'custom_adv3/yaleB36/48.jpeg',
  'custom_adv3/yaleB37/197.jpeg',
  'custom_adv3/yaleB37/494.jpeg',
  'custom_adv3/yaleB37/560.jpeg',
  'custom_adv3/yaleB37/559.jpeg',
  'custom_adv3/yaleB37/564.jpeg',
  'custom_adv3/yaleB37/234.jpeg',
  'custom_adv3/yaleB37/80.jpeg',
  'custom_adv3/yaleB37/465.jpeg',
  'custom_adv3/yaleB37/407.jpeg',
  'custom_adv3/yaleB37/39.jpeg',
  'custom_adv3/yaleB28/503.jpeg',
  'custom_adv3/yaleB28/197.jpeg',
  'custom_adv3/yaleB28/166.jpeg',
  'custom_adv3/yaleB28/248.jpeg',
  'custom_adv3/yaleB28/505.jpeg',
  'custom_adv3/yaleB28/494.jpeg',
  'custom_adv3/yaleB28/560.jpeg',
  'custom_adv3/yaleB28/559.jpeg',
  'custom_adv3/yaleB28/502.jpeg',
  'custom_adv3/yaleB28/281.jpeg',
  'custom_adv3/yaleB17/503.jpeg',
  'custom_adv3/yaleB17/197.jpeg',
  'custom_adv3/yaleB17/494.jpeg',
  'custom_adv3/yaleB17/560.jpeg',
  'custom_adv3/yaleB17/559.jpeg',
  'custom_adv3/yaleB17/502.jpeg',
  'custom_adv3/yaleB17/457.jpeg',
  'custom_adv3/yaleB17/216.jpeg',
  'custom_adv3/yaleB17/234.jpeg',
  'custom_adv3/yaleB17/48.jpeg',
  'custom_adv3/yaleB32/197.jpeg',
  'custom_adv3/yaleB32/492.jpeg',
  'custom_adv3/yaleB32/494.jpeg',
  'custom_adv3/yaleB32/560.jpeg',
  'custom_adv3/yaleB32/339.jpeg',
  'custom_adv3/yaleB32/559.jpeg',
  'custom_adv3/yaleB32/502.jpeg',
  'custom_adv3/yaleB32/281.jpeg',
  'custom_adv3/yaleB32/304.jpeg',
  'custom_adv3/yaleB32/413.jpeg',
  'custom_adv3/yaleB31/197.jpeg',
  'custom_adv3/yaleB31/494.jpeg',
  'custom_adv3/yaleB31/560.jpeg',
  'custom_adv3/yaleB31/559.jpeg',
  'custom_adv3/yaleB31/281.jpeg',
  'custom_adv3/yaleB31/53.jpeg',
  'custom_adv3/yaleB31/325.jpeg',
  'custom_adv3/yaleB31/457.jpeg',
  'custom_adv3/yaleB31/564.jpeg',
  'custom_adv3/yaleB31/216.jpeg',
  'custom_adv3/yaleB16/202.jpeg',
  'custom_adv3/yaleB16/503.jpeg',
  'custom_adv3/yaleB16/197.jpeg',
  'custom_adv3/yaleB16/532.jpeg',
  'custom_adv3/yaleB16/492.jpeg',
  'custom_adv3/yaleB16/494.jpeg',
  'custom_adv3/yaleB16/560.jpeg',
  'custom_adv3/yaleB16/559.jpeg',
  'custom_adv3/yaleB16/502.jpeg',
  'custom_adv3/yaleB16/304.jpeg',
  'custom_adv3/yaleB19/503.jpeg',
  'custom_adv3/yaleB19/197.jpeg',
  'custom_adv3/yaleB19/166.jpeg',
  'custom_adv3/yaleB19/327.jpeg',
  'custom_adv3/yaleB19/492.jpeg',
  'custom_adv3/yaleB19/505.jpeg',
  'custom_adv3/yaleB19/494.jpeg',
  'custom_adv3/yaleB19/560.jpeg',
  'custom_adv3/yaleB19/559.jpeg',
  'custom_adv3/yaleB19/281.jpeg',
  'custom_adv3/yaleB26/197.jpeg',
  'custom_adv3/yaleB26/492.jpeg',
  'custom_adv3/yaleB26/494.jpeg',
  'custom_adv3/yaleB26/281.jpeg',
  'custom_adv3/yaleB26/53.jpeg',
  'custom_adv3/yaleB26/457.jpeg',
  'custom_adv3/yaleB26/216.jpeg',
  'custom_adv3/yaleB26/234.jpeg',
  'custom_adv3/yaleB26/80.jpeg',
  'custom_adv3/yaleB26/465.jpeg']

In [ ]:
len(imageid2ground
   )

In [ ]:
np.save('images_stored.npy', images_stored)


In [ ]:
print(no_faces)

In [ ]:
print(faces)

In [ ]:
print(len(imageid2ground))

In [ ]:
print(len(images_stored))

In [ ]:
print(len(faces))

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/groupface"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)


In [ ]:
task_id = json.loads(response.text)["task_id"]

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
print(response.text)

In [ ]:
from collections import defaultdict

start_token = ''
predicted_clusters = []
ground_truth = []
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"

response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
start_token = json.loads(response.text)["next_token"]
processed = defaultdict(lambda: False)

repeat = defaultdict(int)

while(True):
    print(len(json.loads(response.text)["faces"]))
    
    for j in json.loads(response.text)["faces"]:
        if not processed[j["image_id"]]:
            predicted_clusters.append(int(j["group_id"]))
            ground_truth.append(imageid2ground[j["image_id"]])
            processed[j["image_id"]] = True
        else:
            repeat[j["image_id"]] += 1
        
    try:
        start_token = json.loads(response.text)["next_token"]
    except:
        print(response.text)
        break
        
    response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token": start_token})
        


In [ ]:
repeat

In [ ]:
print(len(predicted_clusters))

print(predicted_clusters.count(0))

In [ ]:
print(len(persons))

In [ ]:
print(len(ground_truth))
print(predicted_clusters.count(0))

In [ ]:
# clas = len(np.unique(ground_truth))
# ground_truth = torch.tensor(ground_truth).cuda()
# predicted_clusters = torch.tensor(predicted_clusters).cuda()

# p = check(ground_truth, predicted_clusters, num_classes=clas, class_names = [0]*clas)
# print(p)

In [ ]:
# print(nmi(ground_truth, predicted_clusters))

In [ ]:
print(np.unique(ground_truth), len(np.unique(ground_truth)))

In [ ]:
print(np.unique(predicted_clusters), len(np.unique(predicted_clusters)))

In [ ]:
# zip(ground_truth, predicted_clusters)

In [ ]:
acc(ground_truth, predicted_clusters)

In [ ]:
nmi(ground_truth, predicted_clusters)

In [ ]:
ari(ground_truth, predicted_clusters)